# Interactive Map of SFO Crime Dataset

### Load the libraries

In [1]:
import numpy as np
import pandas as pd
#Load map library
import folium
from folium import plugins
import matplotlib.pyplot as plt
#Load the interactive library
import ipywidgets as widgets
from ipywidgets import Layout, interactive

### Load the data

In [2]:
data = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")

#Find the unique values of police distict, Incident Category, Day of week.
unique_district = data.PdDistrict.unique()
unique_cat = data.Category.unique()
unique_day = data.DayOfWeek.unique()

### Define the widgets based on the filters/features that are required for the map

In [3]:
style = {'description_width': 'initial'}

# Maximum number of cases is 10000 

limitcase = widgets.IntSlider(
    value=1000,
    min=250,
    max=10000,
    step=1,
    description='Maximum Number of Inc Cases:',
    disabled=False,
    style=style)

# Value specifies the filtered values
pddistrict = widgets.SelectMultiple(
    options = unique_district.tolist(),
    value = ['SOUTHERN','CENTRAL'],
    description='Pd District',
    disabled=False,
    style=style,
    layout = Layout(width='60%', height='90px')
)

Inccategory = widgets.SelectMultiple(
    options = unique_cat.tolist(),
    value = ['VEHICLE THEFT', 'ASSAULT', 'ROBBERY'],
    description='Incident Criminal Case',
    disabled=False,
    style=style,
    layout = Layout(width='60%', height='90px')
)
dayofweek = widgets.SelectMultiple(
    options = unique_day.tolist(),
    value = ['Friday','Saturday','Sunday'],
    description='Day of the week',
    disabled=False,
    style=style,
    layout = Layout(width='60%', height='90px')
)

### Define the map to accept the widgets that you defined

In [4]:
def display_map(district, category, limit, day):
    
    df = data
    df = df.iloc[0:limit, :]
    
    latitude = 37.77
    longitude = -122.42
    
    # define the variable to store corresponding attributes.
    
    df_dist = df.loc[df['PdDistrict'].isin(np.array(district))]
    df_category = df_dist.loc[df_dist['Category'].isin(np.array(category))]
    df_day = df_category.loc[df_category['DayOfWeek'].isin(np.array(day))]
    
    #find the unique values for each attribute/feature.
    
    cat_unique = df_day['Category'].value_counts()
    cat_unique = cat_unique.reset_index()
    
    dist_unique = df_day['PdDistrict'].value_counts()
    dist_unique = dist_unique.reset_index()
    
    day_unique = df_day['DayOfWeek'].value_counts()
    day_unique = day_unique.reset_index()
    
    # Display the plots
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))

    # create map and display it
    sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

    
    incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
    for lat, lng, label, in zip(df_category.Y, df_category.X, df_day.DayOfWeek):
        folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
        ).add_to(incidents)
# Display the map
    display(sanfran_map)
# Display the bar chart for the defined features    
    ax1.bar(cat_unique['index'], cat_unique['Category'])
    ax1.set_title('Number of Criminal Case Based on Category')
    ax2.bar(dist_unique['index'], dist_unique['PdDistrict'])
    ax2.set_title('Number of Criminal Case in Selected District')
    ax3.bar(day_unique['index'], day_unique['DayOfWeek'])
    ax3.set_title('Number of Criminal Case in Selected Day')
    
    plt.show()

### A call for the interactive map by using the widgets

In [5]:
interactive(display_map, district = pddistrict, category = Inccategory, limit=limitcase, day=dayofweek)

interactive(children=(SelectMultiple(description='Pd District', index=(2, 6), layout=Layout(height='90px', wid…

#### Ref : https://github.com/marcellusruben